In [30]:
import re
import sys
from pathlib import Path

import openai
import tiktoken
from dotenv import load_dotenv
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from openai.api_resources import engine
from pqdm.threads import pqdm
from rich.console import Console

load_dotenv()
MODEL_NAME = "gpt-3.5-turbo-16k"
console = Console()
llm = OpenAI(temperature=0)
console.print("Using model", llm.model_name, style="bold green")
text_splitter = CharacterTextSplitter()
CHUNK_SIZE = 4000
OVERLAP = 500

Using model text-davinci-003

In [31]:
def process_document(file: Path):
    content = file.read_text()
    console.print("Processing", file, style="bold yellow")
    console.print("Number of characters:", len(content), style="bold green")

    splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=CHUNK_SIZE,
        chunk_overlap=OVERLAP,
        length_function=len,
    )
    texts = splitter.split_text(content)
    console.print("Number of chunks:", len(texts), style="bold green")
    return texts

In [32]:
file = Path("paper.txt")

In [33]:
texts = process_document(file)

Processing paper.txt

Number of characters: 69701

Number of chunks: 20

In [34]:
embeddings = OpenAIEmbeddings(model=llm.model_name)

In [35]:
embeddings

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-davinci-003', deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=None, openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=6, request_timeout=None, headers=None)

In [36]:
doc_search = FAISS.from_texts(texts, embeddings)

In [37]:
docs = doc_search.similarity_search("What is the meaning of life?")

In [38]:
docs[0].page_content

"The results, presented in Figure , show that once again GPT3 achieves near-perfect performance on in-distribution, but fails entirely in generalizing to OOD cases---in particular, wider or deeper computation graphs. \nThese results indicate that even when training directly with guidance on the computation steps, models still fail to learn component operations in a generalizable manner. This observation holds for all tasks (See details in  ).  \nThese findings suggest that the autoregressive characteristic of Transformers, which forces them to tackle problems sequentially, presents a fundamental challenge that cannot be resolved by instructing the model to generate a step-by-step solution. Instead, models face the fundamental challenge of simply depending on a greedy process of producing the next word to make predictions without a rigorous global understanding of the task. \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\

In [39]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI


In [40]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [41]:
chain.llm_chain.prompt

PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:", template_format='f-string', validate_template=True)

In [42]:
query = "What is the contribution of this work?"
response = chain.run(input_documents=docs,question=query)

In [48]:
import textwrap
print(*textwrap.wrap(response,80),sep="\n")

  This work contributes to the understanding of the successes and failures of
Transformers in compositional tasks by providing a rigorous empirical and
theoretical examination. Specifically, we evaluate the performance of GPT3,
ChatGPT, and GPT4 on our suite of compositional tasks, using zero-shot, few-
shot, and finetuning techniques. We then analyze the results and identify the
challenges that Transformers face in mastering multi-step compositional
problems. We additionally provide theoretical insights on the difficulty of
generalizing these models and suggest empirical strategies for improving them.


In [67]:
from langchain.chains import RetrievalQA

In [ ]:
retriver = RetrievalQA(search_type='similarity', search_kwargs={'k': 5})